In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('dataset\Crop_recommendation.csv')

In [5]:
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [7]:
x= df.drop('label',axis=1)
y=df['label']

In [8]:
x.head()

,N,P,K,temperature,humidity,ph,rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536
1,85,58,41,21.770462,80.319644,7.038096,226.655537
2,60,55,44,23.004459,82.320763,7.840207,263.964248
3,74,35,40,26.491096,80.158363,6.980401,242.864034
4,78,42,42,20.130175,81.604873,7.628473,262.717340


In [9]:
y.sample(5)

1192         mango
1712        papaya
458     pigeonpeas
760      blackgram
2035          jute
Name: label, dtype: object

In [20]:
num_cols = list(x.columns)
num_cols

['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder

In [6]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [21]:
le = LabelEncoder()
y_train_label = le.fit_transform(y_train)
y_test_label = le.transform(y_test)

In [51]:
lst = le.classes_
mapping = {idx:item for idx,item in enumerate(lst)}
mapping

{0: 'apple',
 1: 'banana',
 2: 'blackgram',
 3: 'chickpea',
 4: 'coconut',
 5: 'coffee',
 6: 'cotton',
 7: 'grapes',
 8: 'jute',
 9: 'kidneybeans',
 10: 'lentil',
 11: 'maize',
 12: 'mango',
 13: 'mothbeans',
 14: 'mungbean',
 15: 'muskmelon',
 16: 'orange',
 17: 'papaya',
 18: 'pigeonpeas',
 19: 'pomegranate',
 20: 'rice',
 21: 'watermelon'}

In [27]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_score,recall_score

In [41]:
def evalute_model(y_train,y_train_pred,y_test,y_test_pred):
    acc1 = accuracy_score(y_train,y_train_pred)
    acc2 = accuracy_score(y_test,y_test_pred)
    f1 = f1_score(y_test,y_test_pred,average='macro')
    precision=precision_score(y_test,y_test_pred,average='macro')
    recall= recall_score(y_test,y_test_pred,average='macro')
    print(f"--Trainning Accuracy : {acc1}")
    print("------------Testing------------")
    print(f"- Accuracy : {acc2}")
    print(f"- f1-Score : {f1}")
    print(f"- Precision : {precision}")
    print(f"- Recall : {recall}")
    print("="*50)
    

In [23]:
preprocessing = ColumnTransformer(
    transformers=[
        ('power',PowerTransformer(method='yeo-johnson'),num_cols)
    ]
)

In [25]:
model1 = Pipeline(
    steps=[
        ("col_trans",preprocessing),
        ("svc",SVC(kernel='rbf', gamma= 'auto', degree = 3, C= 100))
    ]
)

In [26]:
model1.fit(x_train,y_train_label)

,steps,"[('col_trans', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('power', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [28]:
y_train_pred1=model1.predict(x_train)
y_test_pred1=model1.predict(x_test)

In [42]:
evalute_model(y_train_label,y_train_pred1,y_test_label,y_test_pred1)

--Trainning Accuracy : 0.9975757575757576
------------Testing------------
- Accuracy : 0.9763636363636363
- f1-Score : 0.9763730630307393
- Precision : 0.9792122909769968
- Recall : 0.9778715891951184


In [43]:
model2 = Pipeline(
    steps=[
        ("col_trans",preprocessing),
        ('randomforest',RandomForestClassifier(n_estimators= 200,min_samples_split= 5,min_samples_leaf= 2,
                        max_depth= 10,criterion= 'entropy',bootstrap= True))
    ]
)

In [44]:
model2.fit(x_train,y_train_label)

,steps,"[('col_trans', ...), ('randomforest', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('power', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [45]:
y_train_pred2=model2.predict(x_train)
y_test_pred2=model2.predict(x_test)

In [46]:
evalute_model(y_train_label,y_train_pred2,y_test_label,y_test_pred2)

--Trainning Accuracy : 0.9993939393939394
------------Testing------------
- Accuracy : 0.9890909090909091
- f1-Score : 0.9887398735692564
- Precision : 0.9900568181818182
- Recall : 0.9891608391608391


In [47]:
model3 = Pipeline(
    steps=[
        ("col_trans",preprocessing),
        ("GB",GradientBoostingClassifier(n_estimators= 100,min_samples_split= 9,min_samples_leaf= 2,
                max_depth= 10,loss= 'log_loss',learning_rate= 0.5,criterion= 'friedman_mse'))
    ]
)

In [48]:
model3.fit(x_train,y_train_label)

,steps,"[('col_trans', ...), ('GB', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('power', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [49]:
y_train_pred3=model3.predict(x_train)
y_test_pred3=model3.predict(x_test)

In [50]:
evalute_model(y_train_label,y_train_pred3,y_test_label,y_test_pred3)

--Trainning Accuracy : 1.0
------------Testing------------
- Accuracy : 0.98
- f1-Score : 0.9796286102575297
- Precision : 0.9803641378674801
- Recall : 0.9814256076404413


#### **Making Pickle files of the models**

In [57]:
import pickle

In [59]:
with open('svc.pkl','wb') as f:
    pickle.dump(model1,f)

In [60]:
with open('rf.pkl','wb') as f:
    pickle.dump(model2,f)

In [61]:
with open('gb.pkl','wb') as f:
    pickle.dump(model3,f)